In [103]:
# Parameters:
patientId = 16361542
dashboard = 1


In [104]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [105]:
#Importing required modules

In [106]:
# import sys
# !{sys.executable} -m pip install --user seaborn
# !{sys.executable} -m pip install --user ipywidgets
# !{sys.executable} -m pip install --user pandas
# !{sys.executable} -m pip install --user matplotlib
# !{sys.executable} -m pip install --user scipy
# !{sys.executable} -m pip install --user sklearn
# !{sys.executable} -m pip install --user fastkml
# !{sys.executable} -m pip install --user folium
# !{sys.executable} -m pip install --user tabulate

from IPython.display import display, clear_output
from IPython.display import Markdown

import IPython.display
import pandas as pd
import numpy as np
import fastai
from fastai.tabular.all import *

import ipywidgets as widgets
from ipywidgets import Button, Layout
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import Box


import warnings
warnings.filterwarnings('ignore')


from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
from pandas.plotting import table

from scipy import stats
from pingouin import ttest
import seaborn as sns
import zipfile
from fastkml.kml import KML
import folium
import datetime
from datetime import date
import sklearn as sklearn
import scikitplot as skplt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler,MinMaxScaler,StandardScaler

from sklearn.ensemble import RandomForestClassifier
from IPython.display import display, HTML
from ipywidgets import widgets, Layout
import scipy.stats as stats
from imblearn.over_sampling import SMOTE
import lightgbm as lgb 
from sklearn.feature_selection import chi2
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from fastai.vision import *

from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

import MySQLdb
from pandas.io import sql 

from simple_colors import *

from sklearn.multioutput import MultiOutputClassifier


In [107]:
# Setting up the database connection
myvars = {}
with open("openmrs-cdss-password.txt") as myfile:
    for line in myfile:
        name, var = line.partition(":")[::2]
        myvars[name.strip()] = var.strip()
        
conn = MySQLdb.connect(host="localhost", user=myvars['DB username'], passwd=myvars['DB password'], db=myvars['DB databasename']) 
openmrscursor = conn.cursor()  

In [108]:
## Patient Attributes
## 12576264, 10152346
patient_id = patientId
values_mapx = {}
values_mapx['patient_id'] = patient_id

In [109]:
dataset=pd.read_csv("merged_modified_named.csv")
dataset=dataset.drop(["Unnamed: 0"],axis=1)
dataset=dataset.drop_duplicates()
dataset=dataset.reset_index()
dataset=dataset.drop(["index"],axis=1)
# Rename some columns
dataset = dataset.rename(columns={
    '1': 'Acute kidney failure, unspecified',
    '2_y': 'Coronary atherosclerosis of native coronary artery',
    '3_y': 'Urinary tract infection, site not specified',
    '4_y': 'Encounter for antineoplastic chemotherapy',
    '5_y': 'Syncope and collapse',
    '6_y': 'Other chest pain',
    '7': 'Pneumonia, organism unspecified',
    '8_y': 'Chest pain, unspecified',
    '9_y': 'Percutaneous transluminal coronary angioplasty [PTCA]',
    '10': 'Other nonoperative respiratory measurements',
})
#dataset

In [110]:
from IPython.core.display import display, HTML
display(HTML('''
<style>
.select-checkbox option::before {
  font-family: 'Material Symbols Outlined';
  font-size: initial;
  content: "\\e835";
  width: 1.5em;
  text-align: center;
  vertical-align: bottom;
  display: inline-block;
}

.select-checkbox option:checked::before {
  content: "\\e834";
}
</style>

<link href="https://fonts.googleapis.com/css2?family=Material+Symbols+Outlined" rel="stylesheet" />
'''))

In [111]:
# creating cohort
def on_1button_clicked(selected_diag):
     # Convert selected_diag to a list and remove any empty elements
    diag_con = list(filter(None, selected_diag))
    values_mapx['diag_con'] = diag_con

    # Select the columns in the DataFrame that have the same values as diag_con
    selected_cols = dataset.loc[:, dataset.columns.isin(diag_con)]

    # Count the number of rows where the selected columns have value 0
    count_rows = selected_cols.ne(0).all(axis=1).sum()
    
    cohort_count = widgets.IntText(value=count_rows, description='Patients similar to selected diagnosis:', style={'description_width': 'initial', 'width': 'auto'})
    
#     item_layout = widgets.Layout(display='flex',
#                     flex_flow='row',
#                     align_items='stretch',
#                     border='solid #1ca999 1px',
#                     padding='1px',
#                     width='100%')
    item_layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid #1ca999 1px',
                    padding='1px',
                    width='100%')

    input_widgets = widgets.HBox([cohort_count], layout=item_layout)
    display(input_widgets)

In [112]:
style = {'description_width': 'initial'}
diagnames_pat_id = "SELECT description from concept where concept_id IN (SELECT * FROM(SELECT diagnosis_coded FROM encounter_diagnosis WHERE patient_id = {patient_id}) as xyz)" 
diagnames_pat_id = diagnames_pat_id.format(**values_mapx)
diagnames_pat_id = pd.read_sql(diagnames_pat_id, con=conn)
diagnames_value = diagnames_pat_id['description']

list_of_diagnosis=['Acute kidney failure, unspecified','Coronary atherosclerosis of native coronary artery','Urinary tract infection, site not specified',
                  'Encounter for antineoplastic chemotherapy','Syncope and collapse','Other chest pain','Pneumonia, organism unspecified',
                  'Chest pain, unspecified','Percutaneous transluminal coronary angioplasty [PTCA]','Other nonoperative respiratory measurements']

diagnames_options = [d for d in diagnames_pat_id['description'].unique() if d in list_of_diagnosis]
dropdown_diagnames_pat_id = widgets.SelectMultiple(options=diagnames_options, style=style, disabled=False, rows=7, size=10, layout=Layout(width='60%')).add_class('select-checkbox')

output_diagnames_pat_id = widgets.Output()

def dropdown_diagconcept_eventhandler(change):
    output_diagnames_pat_id.clear_output()
    with output_diagnames_pat_id:
        display(diagnames_pat_id[diagnames_pat_id['description'] == change.new])
    on_1button_clicked(dropdown_diagnames_pat_id.value)
     
dropdown_diagnames_pat_id.observe(dropdown_diagconcept_eventhandler, names='value')

In [113]:
patient_attribute_df = pd.read_sql("""select p.gender, m.age_group_code, c.concept_id, o.concept_id, cn.name,  c.description, 
                                e.diagnosis_coded from person as p LEFT JOIN mimic_openmrs_cdss as m ON p.person_id = m.person_id 
                                LEFT JOIN obs as o ON o.person_id = p.person_id  LEFT JOIN concept_name as cn ON cn.concept_id = o.concept_id 
                                LEFT JOIN encounter_diagnosis as e ON e.patient_id = p.person_id LEFT JOIN concept as c ON 
                                c.concept_id = e.diagnosis_coded WHERE  p.person_id = {}""".format(patient_id), con = conn)

In [114]:
def on_3button_clicked():
    output_count = widgets.Output()
    concept_name = dropdown_concept_name.value
    concept_name = list(concept_name)

    values_map = {}

    a = 0
    final_patient_ind_value_numeric = pd.DataFrame(columns = ['person_id'])
    final_patient_ind_value_numeric['person_id'] = final_patient_ind_value_numeric['person_id'].astype(int)

    dfss = []

    for i in concept_name:
        values_map['concept_name'] = concept_name[a]
        values_map['patient_id'] = patient_id 
        a = a+1

        patient_value_numeric = (""" select o.person_id, o.value_numeric from obs as o where o.concept_id IN (SELECT c.concept_id FROM concept as c WHERE c.description = '{concept_name}') AND o.person_id = {patient_id}""")
        patient_value_numeric = patient_value_numeric.format(**values_map)

        patient_ind_value_numeric = pd.read_sql(patient_value_numeric, con = conn)
    
        patient_ind_value_numeric.columns = [*patient_ind_value_numeric.columns[:-1], i]
    
        name = "patient_ind_value_numeric" + i
     
        locals()[name] = patient_ind_value_numeric.copy()

        dfss.append(locals()[name])

    ls = [df.set_index('person_id') for df in dfss]

    # pd.DataFrame.join accepts a list of dataframes as 'other'
    final_patient_ind_value_numeric = ls[0].append(ls[1:])
    
    # final_patient_ind_value_numeric = ls[0].join(ls[1:], how = 'inner')
    final_patient_ind_value_numeric = final_patient_ind_value_numeric.reset_index(drop = False)

    for column in final_patient_ind_value_numeric.columns:
        final_patient_ind_value_numeric[column] = final_patient_ind_value_numeric[column].fillna(final_patient_ind_value_numeric[column].mean())

    display(final_patient_ind_value_numeric)

In [115]:
def patient_cohort_concept_():
    output_count = widgets.Output()

    concept_name = dropdown_concept_name.value
    concept_name = list(concept_name)
    concept_name

    values_map = {}
    a = 0
    
    # final_patient_cohort_concept = pd.DataFrame(columns = ['person_id'])
    # final_patient_cohort_concept['person_id'] = final_patient_cohort_concept['person_id'].astype(int)

    dfs = []

    for i in concept_name:
        values_map['concept_name'] = concept_name[a]
        values_map['patient_id'] = patient_id 
        a = a+1
    
        sql_query = ("""select person_id, value_numeric from obs where person_id IN (select p.person_id from person p, 
                mimic_openmrs_cdss cdss where p.person_id IN (select patient_id from encounter_diagnosis where 
                diagnosis_coded IN (select c.concept_id from concept c where c.description LIKE 
                ('Pneumonia, unspecified organism')) AND cdss.person_id=p.person_id AND p.gender='M' 
                AND cdss.age_group_code=7)) AND concept_id = (select c.concept_id from concept c where c.description = '{concept_name}')""")

        sql_query = sql_query.format(**values_map)

        patient_cohort_concept = pd.read_sql(sql_query, con = conn)
        patient_cohort_concept.columns = [*patient_cohort_concept.columns[:-1], i]
    
        name = "patient_cohort_concept" + i
    
 
        locals()[name] = patient_cohort_concept.copy()

        dfs.append(locals()[name])
    
    
   
    #final_patient_cohort_concept.merge(patient_cohort_concept, on = 'person_id', how = 'outer')
    #final_patient_cohort_concept = pd.merge(patient_cohort_concept, final_patient_cohort_concept, on='person_id', how='inner')
    #l = [df.set_index('person_id') for df in dfs] 

    l = [df for df in dfs] 

    # pd.DataFrame.join accepts a list of dataframes as 'other' 
    final_patient_cohort_concept = l[0].append(l[1:])


    #final_patient_cohort_concept = l[0].append(l[1:])
    #final_patient_cohort_concept = final_patient_cohort_concept.reset_index(drop = False)
    for column in final_patient_cohort_concept.columns:
        final_patient_cohort_concept[column] = final_patient_cohort_concept[column].fillna(final_patient_cohort_concept[column].mean())

        out = widgets.Output()
        with out:
            display(final_patient_cohort_concept)
        return out

    # Getting total count of the patient cohort as text widget
    total_count_cohort = len(final_patient_cohort_concept)
    text_patient_cohort_count = widgets.IntText(value = total_count_cohort , description= 'Total count:')
    display(text_patient_cohort_count)

In [116]:
# getting the dropdown for concept_name

In [117]:
output_concept_name = widgets.Output()
#dropdown_concept_name = widgets.Dropdown(options = patient_attribute_df['description'].unique(), description = '<b>Concept Name:</b>')


columns_list = list(dataset.columns)[:423]
# create a new DataFrame containing only the relevant rows from patient_attribute_df
relevant_rows = patient_attribute_df[patient_attribute_df['name'].isin(columns_list)]

dropdown_concept_name = widgets.SelectMultiple(options= relevant_rows['name'].unique(), style =style,disabled=False, rows = 7, size = 10,layout=Layout(width = '40%')).add_class('select-checkbox')


def dropdown_concept_name_eventhandler(change):
    output_concept_name.clear_output()
    with output_concept_name:
        display(patient_attribute_df[patient_attribute_df["name"] == change.new])
    patient_cohort_concept_()
    on_3button_clicked(dropdown_concept_name.value)
    
def dropdown_concept_name_eventhandler1(change):
    output_concept_name.clear_output()
    with output_concept_name:
        display(patient_attribute_df[patient_attribute_df["name"] == change.new])

    
dropdown_concept_name.observe(dropdown_concept_name_eventhandler1, names='value')

In [118]:
# getting values of the concept name of the cohort

In [119]:
def conc():
    output_count = widgets.Output()
    concept_name = dropdown_concept_name.value
    concept_name = list(concept_name)
    concept_name.append('patient_id')

    # Filter columns based on similarity to concept_name
    similar_columns = [col for col in dataset.columns if col in concept_name]

    # Create a new DataFrame with only the similar columns
    final_patient_cohort_concept = dataset[similar_columns]
    final_patient_cohort_concept = final_patient_cohort_concept.rename(columns={'patient_id': 'person_id'})
    # Drop rows with missing values (NaN)
    final_patient_cohort_concept = final_patient_cohort_concept.dropna()
    for column in final_patient_cohort_concept.columns:
        final_patient_cohort_concept[column] = final_patient_cohort_concept[column].fillna(final_patient_cohort_concept[column].mean())
    
        def patient_cohort_concept_():
            out = widgets.Output()
            with out:
                display(final_patient_cohort_concept)
            return out
    return final_patient_cohort_concept

In [120]:
## write a function to append each concepts and its values

In [121]:
# get individual patient value

In [122]:
def con():
    
    output_count = widgets.Output()

    concept_name = dropdown_concept_name.value
    concept_name = list(concept_name)

    # concept_name = ['creatinine', 'albumin']
    values_map = {}

    a = 0
    final_patient_ind_value_numeric = pd.DataFrame(columns = ['person_id'])
    final_patient_ind_value_numeric['person_id'] = final_patient_ind_value_numeric['person_id'].astype(int)

    dfss = []

    for i in concept_name:
        values_map['concept_name'] = concept_name[a]
        values_map['patient_id'] = patient_id 
        a = a+1

        patient_value_numeric = (""" select o.person_id, o.value_numeric from obs as o where o.concept_id IN (SELECT concept_id FROM concept WHERE description = '{concept_name}') AND o.person_id = {patient_id}""")
        patient_value_numeric = patient_value_numeric.format(**values_map)

        patient_ind_value_numeric = pd.read_sql(patient_value_numeric, con = conn)

        patient_ind_value_numeric.columns = [*patient_ind_value_numeric.columns[:-1], i]

        name = "patient_ind_value_numeric" + i

        locals()[name] = patient_ind_value_numeric.copy()

        dfss.append(locals()[name])

    ls = [df.set_index('person_id') for df in dfss]

    # pd.DataFrame.join accepts a list of dataframes as 'other'
    final_patient_ind_value_numeric = ls[0].append(ls[1:])
    # final_patient_ind_value_numeric = ls[0].join(ls[1:], how = 'inner')
    final_patient_ind_value_numeric = final_patient_ind_value_numeric.reset_index(drop = False)

    for column in final_patient_ind_value_numeric.columns:
        final_patient_ind_value_numeric[column] = final_patient_ind_value_numeric[column].fillna(final_patient_ind_value_numeric[column].mean())

    return final_patient_ind_value_numeric

In [123]:
# Plot for Normal Distribution

In [124]:
def showplot():
    final_patient_ind_value_numeric=con()
    final_patient_cohort_concept=conc()
    concept_name = dropdown_concept_name.value
    plot_output = widgets.Output()
    plot_output.clear_output()
    concept_name = list(concept_name)
    tab = widgets.Tab()
    
    out = []
    with plot_output:
        num_plots = len(concept_name)
        fig, axs = plt.subplots(nrows=(num_plots+1)//2, ncols=min(num_plots, 2), figsize=(15, 5*(num_plots+1)//2))
        axs = np.array(axs).flatten()
        for i, column in enumerate(concept_name):
            sns.kdeplot(data=final_patient_cohort_concept[column], color='crimson', label='cohort', fill=True, ax=axs[i])
            sns.kdeplot(data=final_patient_ind_value_numeric[column], color='limegreen', label='patient', fill=True, ax=axs[i])
            axs[i].legend(labels=['cohort', 'patient'])
            axs[i].set_title("Normality Distribution")
            axs[i].set_xlabel(column)

        for j in range(i+1, len(axs)):
            axs[j].remove()
        plt.tight_layout()
        plt.show()
            
    display(plot_output)
#showplot()

In [125]:
## checking mean of diagnosis for each patient

In [126]:
encounter_diagnosis = pd.read_sql('select patient_id, diagnosis_coded from encounter_diagnosis', con=conn)
encounter_diagnosis_count = encounter_diagnosis.groupby("patient_id")["diagnosis_coded"].count()

In [127]:
top_20_diag = pd.DataFrame(encounter_diagnosis['diagnosis_coded'].value_counts().sort_index(ascending=False).sort_values(ascending=False).head(25))
top_20_diag = top_20_diag.reset_index()
list_top_20_diag = list(top_20_diag['index'])
#list_top_20_diag

In [128]:
## Obtaining class labels ##

In [129]:
class_labels = pd.read_sql("SELECT description from concept where concept_id IN (SELECT diagnosis_coded FROM(SELECT diagnosis_coded FROM encounter_diagnosis WHERE diagnosis_coded IN ('1385187','1385109','1384870','1384841','1384995','1397715','1384886','1474651','1385083','1385121'))as xyz)", con=conn) 
#class_labels                      

In [130]:
old_diagnosis_name = None
#run this cell only when we restart the kernel

In [131]:
#Dashboard Layout

In [132]:
from IPython.display import display, HTML
from ipywidgets import widgets, Layout

BStep1 = widgets.Button(
    description='Step 1: Please select ONE or MULTIPLE Diagnosis',
    layout=Layout(width='1635px'),
    style={'button_color': '#1ca999'},
    tooltip='Step 1'
)

BStep2 = widgets.Button(
    description='Step 2: Please select ONE or MULTIPLE Concept Names',
    layout=Layout(width='1070px'),
    style={'button_color': '#1ca999'},
    tooltip='Step 2'
)

item_layout = widgets.Layout(
    display='flex',
    flex_flow='row',
    align_items='stretch',
    border='solid 1px #1ca999',
    width='100%',
    justify_content='space-between'
)

steps = widgets.Box([BStep1, BStep2], layout=item_layout)

button_style = '<style>.my-button { color: white; }</style>'
display(HTML(button_style))
BStep1.add_class("my-button")
BStep2.add_class("my-button")

display(steps)


Box(children=(Button(description='Step 1: Please select ONE or MULTIPLE Diagnosis', layout=Layout(width='1635p…

In [133]:
def buttons():
    cdbutton=widgets.Button(description='Choose Diagnosis', layout=Layout(width='520px'), tooltip='Show Normal Distribution',text_color='white')
    cnbutton=widgets.Button(description='Concept Name', layout=Layout(width='520px'), tooltip='Perform Corelation',text_color='white')
    buttons=widgets.HBox([cdbutton,cnbutton])
    display(buttons)

In [134]:
item_layout = widgets.Layout(display='flex',
                    align_items='stretch',
                    border='solid #1ca999 1px',
                    padding='left',
                    width='100%')
input_widgets = widgets.HBox([dropdown_diagnames_pat_id, dropdown_concept_name], layout = item_layout)
display(input_widgets)

In [135]:
output_diag_name = widgets.Output()

dropdown_diagnosis_name = widgets.Dropdown(options= class_labels['description'].unique(), style = style, description='Risk of diagnosis for the patient:',disabled=False, rows = 15, size = 10, layout=Layout(width = '450px'))

def dropdown_diagnosis_name_eventhandler(change):
#     output_diag_name.clear_output()
#     with output_diag_name:
#         display(class_labels[class_labels["description"] == change.new])
#         diagnosis_name = dropdown_diagnosis_name.value    
#     dash()
    output_diag_name.clear_output()
    with output_diag_name:
        display(class_labels[class_labels["description"] == change.new])
        diagnosis_name = dropdown_diagnosis_name.value  
    button = widgets.Button(description="Show Detailed Results")
    button.on_click(show_tabs)
    multiclass()
        
# dropdown_diagnosis_name.observe(dropdown_diagnosis_name_eventhandler, names='value')
        
dropdown_diagnosis_name.observe(dropdown_diagnosis_name_eventhandler, names='value')  

In [136]:
button_style = '<style>.my-button { color: white; background-color: #1ca999; }</style>'
BStep3 = widgets.Button(description='Step 3: Please select ONE risk of diagnosis for the patient', 
                        layout=Layout(width='2000px'), 
                        style={'description_width': 'initial', 'button_width':'100%'}, 
                        tooltip='Step 2')
display(HTML(button_style))
BStep3.add_class("my-button")
item_layout = widgets.Layout(display='center',
                    flex_flow='center',
                    align_items='stretch',
                    border='solid #1ca999 1px',
                    width='100%')
input_step3 = widgets.HBox([BStep3], layout=item_layout)
display(input_step3)

In [137]:
item_layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    justify_content='center',
                    border='solid #1ca999 1px',
                    padding='1px',
                    width='100%')

input_widgets = widgets.HBox([dropdown_diagnosis_name], layout = item_layout)
display(input_widgets)

In [138]:
def risk():

    BStep1 = widgets.Button(
        description='Step 1: Please select ONE or MULTIPLE Diagnosis',
        layout=Layout(width='1635px'),
        style={'button_color': '#1ca999'},
        tooltip='Step 1'
    )

    BStep2 = widgets.Button(
        description='Step 2: Please select ONE or MULTIPLE Concept Names',
        layout=Layout(width='1070px'),
        style={'button_color': '#1ca999'},
        tooltip='Step 2'
    )

    item_layout = widgets.Layout(
        display='flex',
        flex_flow='row',
        align_items='stretch',
        border='solid 1px #1ca999',
        width='100%',
        justify_content='space-between'
    )

    steps = widgets.Box([BStep1, BStep2], layout=item_layout)

    button_style = '<style>.my-button { color: white; }</style>'
    display(HTML(button_style))
    BStep1.add_class("my-button")
    BStep2.add_class("my-button")

    display(steps)
    item_layout = widgets.Layout(display='flex',
                    align_items='stretch',
                    border='solid #1ca999 1px',
                    padding='left',
                    width='100%')
    input_widgets = widgets.HBox([dropdown_diagnames_pat_id, dropdown_concept_name], layout = item_layout)
    display(input_widgets)
    button_style = '<style>.my-button { color: white; background-color: #1ca999; }</style>'
    BStep3 = widgets.Button(description='Step 3: Please select ONE risk of diagnosis for the patient', 
                            layout=Layout(width='2000px'), 
                            style={'description_width': 'initial', 'button_width':'100%'}, 
                            tooltip='Step 2')
    display(HTML(button_style))
    BStep3.add_class("my-button")
    item_layout = widgets.Layout(display='center',
                        flex_flow='center',
                        align_items='stretch',
                        border='solid #1ca999 1px',
                        width='100%')
    input_step3 = widgets.HBox([BStep3], layout=item_layout)
    display(input_step3)

    item_layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    justify_content='center',
                    border='solid #1ca999 1px',
                    padding='1px',
                    width='100%')

    input_widgets = widgets.HBox([dropdown_diagnosis_name], layout = item_layout)
    display(input_widgets)

In [139]:
#Creating Tabs

In [140]:
import sys
sys.setrecursionlimit(10000)  # set maximum recursion depth to 10000
from IPython.display import clear_output

def dash():
    # create tabs
    tab1 = widgets.Output()
    tab2 = widgets.Output()
    tab3 = widgets.Output()
#     tab4 = widgets.Output()

    tab = widgets.Tab(children=[tab1, tab2, tab3])
    tab.set_title(0, 'Distribution Plot')
    tab.set_title(1, 'Statistical Test')
    tab.set_title(2, 'Correlation Test')
#     tab.set_title(3, 'Association Test')
    display(tab)

    # clear the tabs before assigning new outputs
    tab1.clear_output()
    tab2.clear_output()
    tab3.clear_output()
#     tab4.clear_output()

    # add the functions to the corresponding tabs
    with tab1:
        showplot()

    with tab2:
        diagnosis_name = dropdown_diagnosis_name.value
        shapiro_output = check_normality(diagnosis_name)
        display(shapiro_output)

    with tab3:
        diagnosis_name = dropdown_diagnosis_name.value
        corr_output = on_corrButton_clicked(diagnosis_name)
        display(corr_output)



In [141]:
def show_tabs(button):
    dash()

In [142]:
## Adding class labels to cohort dataframe
#final_patient_cohort_concept[diagnosis_name] = ''

def final_patient_cohort_concept_1(diagnosis_name):
    diagnosis_name = [dropdown_diagnosis_name.value]  # Convert to a list
    values_map3 = {'diagnosis_name': diagnosis_name}

    selected_cols = dataset.loc[:, dataset.columns.isin(diagnosis_name)]
    get_patient_id = dataset.loc[selected_cols.ne(0).any(axis=1), 'patient_id']

    
    diagnosis_name = dropdown_diagnosis_name.value
    final_patient_cohort_concept=conc()

    for i, row in final_patient_cohort_concept.iterrows():
        if row['person_id'] in get_patient_id['patient_id'].tolist():

            final_patient_cohort_concept.loc[i, [diagnosis_name]] = 1
        else:
            final_patient_cohort_concept.loc[i, [diagnosis_name]] = 0
            
    return final_patient_cohort_concept   

In [143]:
diagnosis_name = dropdown_diagnosis_name.value
if old_diagnosis_name != diagnosis_name:
    try:
        final_patient_cohort_concept=final_patient_cohort_concept_1(diagnosis_name)
        final_patient_cohort_concept.drop([old_diagnosis_name], axis =1, inplace=True)
    except Exception as ex:
        #print(ex)
        pass
    old_diagnosis_name = diagnosis_name

In [144]:
#final_patient_cohort_concept_feat = final_patient_cohort_concept.drop(columns = ['person_id'])
# final_patient_cohort_concept=final_patient_cohort_concept_1(diagnosis_name)
# #final_patient_cohort_concept_feat

In [145]:
diagnosis_name = [dropdown_diagnosis_name.value]  # Convert to a list
values_map3 = {'diagnosis_name': diagnosis_name}

selected_cols = dataset.loc[:, dataset.columns.isin(diagnosis_name)]
get_patient_id = dataset.loc[selected_cols.ne(0).any(axis=1), 'patient_id']

# Create a DataFrame with column name 'patient_id'
get_patient_id = pd.DataFrame({'patient_id': get_patient_id})
# get_patient_id

In [146]:
## Correlation Test

In [147]:
def on_corrButton_clicked(diagnosis_name):
    diagnosis_name = dropdown_diagnosis_name.value
    final_patient_cohort_concept=conc()
    concept_name = dropdown_concept_name.value


    
    for i, row in final_patient_cohort_concept.iterrows():
        if row['person_id'] in get_patient_id['patient_id'].tolist():
      
            final_patient_cohort_concept.loc[i, [diagnosis_name]] = 1
        else:
            final_patient_cohort_concept.loc[i, [diagnosis_name]] = 0
    
    final_patient_cohort_concept_feat = final_patient_cohort_concept.drop(columns = ['person_id'])
    corr_output=widgets.Output()
    
    with corr_output:

        plt.figure(figsize = (6,5))
        corr = final_patient_cohort_concept_feat.corr(method='spearman')
        ax = sns.heatmap(corr, annot=True, fmt ='.1g', vmin=-1, vmax=1, center= 0, cmap= 'coolwarm')   
        
        plt.show(ax)
    display(corr_output)

In [148]:
#Association Test (Shapiro-Wilk Test Statistic(T test and Mann Whitney U test))

In [149]:
# Function to check normality and perform appropriate tests
def check_normality(diagnosis_name):
    # Perform Shapiro-Wilk test
    final_patient_ind_value_numeric = con()
    final_patient_cohort_concept = conc()
    concept_name = dropdown_concept_name.value
    
    shapiro_output = widgets.Output()
    with shapiro_output:
        
        table_data = []
        for j in concept_name:
            shapiro = stats.shapiro(final_patient_cohort_concept[j])
            if shapiro.pvalue >= 0.05:
                test_name = 'T Test'
                test_result = stats.ttest_ind(final_patient_cohort_concept[j], final_patient_ind_value_numeric[j], equal_var=False)
            else:
                test_name = 'Mann-Whitney U test'
                test_result = stats.mannwhitneyu(final_patient_cohort_concept[j], final_patient_ind_value_numeric[j], alternative='two-sided')
            
            table_data.append([j, test_name, test_result.statistic, test_result.pvalue])
        
        column_names = ['Concept', 'Test', 'Statistic', 'p-value']
        df = pd.DataFrame(table_data, columns=column_names)
        
        # Create HTML table with borders and centered text
        html_table = df.to_html(index=False)
        html_table = html_table.replace('<table', '<table style="width:100%; table-layout:fixed; border-collapse: collapse;"')
        html_table = html_table.replace('<th>', '<th style="text-align:center; vertical-align:middle; border: 1px solid black;">')
        html_table = html_table.replace('<td>', '<td style="text-align:center; vertical-align:middle; border: 1px solid black;">')
        
        # Add CSS styling to increase column width and align text
        html_table = html_table.replace('<caption>', '<caption style="caption-side: top; text-align: center; font-weight: bold;">')
        
        display(HTML(html_table))
    
    return shapiro_output


In [150]:
dataset_feb04=pd.read_csv("merged_modified2.csv")
dataset_feb04=dataset_feb04.drop(["Unnamed: 0"],axis=1)
dataset_feb04=dataset_feb04.drop_duplicates()
dataset_feb04=dataset_feb04.reset_index()
dataset_feb04=dataset_feb04.drop(["index"],axis=1)
# # dataset_feb04

In [151]:
#Multiclassoutput

In [152]:
def multiclass():
#     dropdown_concept_name = patient_attribute_df['name'].unique()
#     concept_name = list(dropdown_concept_name)
    output_count = widgets.Output()
    concept_name = dropdown_concept_name.value
    concept_name = list(concept_name)
    data_dff = pd.DataFrame()

    for i in range(len(concept_name)):
        values_map = {'concept_name': concept_name[i], 'patient_id': patient_id}

        patient_value_numeric = ("""SELECT o.concept_id, o.person_id, o.value_numeric FROM obs AS o WHERE o.concept_id IN (SELECT c.concept_id FROM concept AS c WHERE c.description = '{concept_name}') AND o.person_id = {patient_id}""")
        patient_value_numeric = patient_value_numeric.format(**values_map)

        patient_ind_value_numeric = pd.read_sql(patient_value_numeric, con=conn)

        if not patient_ind_value_numeric.empty:
            concept_id = patient_ind_value_numeric["concept_id"].iloc[0]
            patient_ind_value_numeric = patient_ind_value_numeric["value_numeric"].mean()

            dff = pd.DataFrame({concept_id: [patient_ind_value_numeric]})
            data_dff = pd.concat([data_dff, dff], axis=1)


    data_dff["patient_id"] = patient_id
    data_dff = data_dff.dropna(axis=1, how='all')
    data_dff = data_dff.groupby("patient_id").max()
    data_dff['1_y'] = 0
    data_dff['2'] = 0
    data_dff['3_y'] = 0
    data_dff['4_y'] = 0
    data_dff['5'] = 0
    data_dff['6_y'] = 0
    data_dff['7'] = 0
    data_dff['8_y'] = 0
    data_dff['9'] = 0
    data_dff['10_y'] = 0
    data_dff=data_dff.reset_index()
    data_dff
#     display(data_dff)
    
    test_set=data_dff
    test_set=test_set.replace("NaN",np.nan)
    test_set=test_set.dropna(axis=1, how='all')
    test_set.columns=test_set.columns.astype(str)

    list_columns = test_set.columns.tolist()
    data1 = dataset_feb04[list_columns]
    training_set = data1.query("patient_id != @patient_id")
    training_set=training_set.dropna(axis=0)
    row_count = len(training_set)
    style = {'description_width': 'unset','width': '1700px'}

    st = time.time()

    prediction_arr=[]
    y_test_arr=[]    
    n_columns=len(test_set.columns)
    X_test=test_set.iloc[:,1:n_columns-10]
    X_test = X_test.dropna(how='all', axis=1)
    

    n_col=len(X_test.columns)
#     print("Number of features found for Patient", n_col)
    num_feat= widgets.Text(value = str(n_col), description= 'Number of features found for Patient:', style =style)
#     display(num_feat)


    n_columns=len(training_set.columns)
    y_train=training_set.iloc[:,n_columns-10:n_columns]
    y_train=y_train.replace({2:1,3:1,4:1,5:1,6:1,7:1,8:1,9:1,10:1},regex=True)

    y_train=y_train.values


    X_train=training_set.iloc[:,1:n_columns-10]
    #display(X_train)
#     print("Number of patients in Patient Cohort:", X_train.shape[0])
    num_cohort= widgets.Text(value = str(row_count), description= 'Number of patients in Patient Cohort:', style =style)
#     display(num_cohort)
    
    if X_train.shape[0]>=1:
        n_samples, n_features = X_train.shape # 10000,605
        n_outputs = y_train.shape[1] # 10
        n_classes = 11

        TOTAL_TREES=int(n_col/3)
        if TOTAL_TREES>64:
            TOTAL_TREES=TOTAL_TREES
        else:
            TOTAL_TREES=64
    else:
          print("Number of patients found in the cohort for same concepts for this patient is less than 1, ignoring training model")

    
    list_of_diagnosis=['Acute kidney failure, unspecified','Coronary atherosclerosis of native coronary artery','Urinary tract infection, site not specified',
                      'Encounter for antineoplastic chemotherapy','Syncope and collapse','Other chest pain','Pneumonia, organism unspecified',
                      'Chest pain, unspecified','Percutaneous transluminal coronary angioplasty [PTCA]','Other nonoperative respiratory measurements']
    
    for i in range(len(list_of_diagnosis)):
        diagnosis_name = dropdown_diagnosis_name.value
        selected_diagnosis_name=diagnosis_name
        if selected_diagnosis_name==list_of_diagnosis[i]:
            if i==0 or i==1 or i==2 or i==5:
                model= widgets.Text(value = str('Random Forest Model'), description= 'Model Trained:', style =style)
#                 display(model)
                forest = RandomForestClassifier(n_estimators=TOTAL_TREES, random_state=1234, n_jobs=-1)
                multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)
                trained_model=multi_target_forest.fit(X_train, y_train)
                prediction=trained_model.predict(X_test)
                prediction_probability=trained_model.predict_proba(X_test)
                break
            else:
                model= widgets.Text(value = str('Light GBM Model'), description= 'Model Trained:', style =style)
#                 display(model)
                forest = lgb.LGBMClassifier()
                multi_target_forest = MultiOutputClassifier(forest, n_jobs=2)
                prediction=multi_target_forest.fit(X_train, y_train).predict(X_test)
                prediction_probability=multi_target_forest.predict_proba(X_test)
                break


#     print(diagnosis_name)
    #print(prediction_probability)
    probability_of_diagnosis=prediction_probability[i][0][1]*100 
    probability_of_diagnosis=round(probability_of_diagnosis,3)
    report= widgets.Text(value = str(probability_of_diagnosis)+"%", description= 'Probability of selected diagnosis:', style =style)
    button_style = '<style>.my-button { color: white; background-color: #1ca999; }</style>'
    stats_button = widgets.Button(description="Show Statistical Tests")
    display(HTML(button_style))
    stats_button.add_class("my-button")
    
    stats_button.on_click(show_tabs)

    # create reset button
    button_style = '<style>.my-button { color: white; background-color: #1ca999; }</style>'
    reset_button = widgets.Button(description="Reset")
    display(HTML(button_style))
    reset_button.add_class("my-button")
    
    # Define a function to reset the output generated by the risk() function

    from IPython.display import Javascript


    def reset_output(button):
        display(Javascript('''
            var cells = Jupyter.notebook.get_cells();
            for (var i = 0; i < cells.length; i++) {
                cells[i].clear_output();
            }
        '''))
        risk()


    # Set the reset_output function to be called when the reset_button is clicked
    reset_button.on_click(reset_output)


    # Create the input widgets and button row
    item_layout = widgets.Layout(display='flex',
                                  align_items='stretch',
                                  border='solid #1ca999 1px',
                                  padding='left',
                                  width='100%',
                                  flex_flow='row wrap',
                                  justify_content='center')

    col1 = widgets.VBox([num_feat, model])
    col2 = widgets.VBox([num_cohort, report])
    input_widgets = widgets.HBox([col1, col2], layout=item_layout)
    button_row = widgets.HBox([stats_button, reset_button], layout=item_layout)

    # Create a VBox container to hold the input_widgets and button_row
    all_widgets = widgets.VBox([input_widgets, button_row])

    # Display the VBox container and the output widget
    display(all_widgets)

    
    # display the widgets
#     display(grid)
    # get the end time
    et = time.time()

    # get the execution time
    elapsed_time = et - st
#     print('Execution time:', elapsed_time, 'seconds')
#     return i,prediction_probability

In [153]:
# from IPython.core.display import display, HTML
# display(HTML('''
# <script>
# $('option').mousedown(function(e) {
#     e.preventDefault();
#     $(this).prop('selected', !$(this).prop('selected'));
#     return false;
# });
# </script>
# '''))